In [ ]:
import datashader as ds
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs

from holoviews.operation.datashader import datashade, rasterize
from colorcet import cm_n
from earthsim.io import read_3dm_mesh, read_mesh2d

datashade.precompute = True

hv.extension('bokeh')
%output holomap='scrubber' fps=2
%opts Image RGB [width=800 height=600]

## Static meshes

Static trimeshes can be loaded using the ``read_3dm_mesh`` utility, which will return its simplices and vertices. Here we will load a mesh of the Chesapeake and Delaware Bays.

Before we declare the ``TriMesh`` we will project the coordinates of the vertices to a Mercator projection for plotting, doing this ahead of time will prevent GeoViews from projecting the coordinates whenever the plot is displayed. We also declare the 'z' dimension representing the values at each vertex.

In [ ]:
fpath = 'Example_Meshes/Chesapeake_and_Delaware_Bays.3dm'
tris, verts = read_3dm_mesh(fpath)
points = gv.operation.project_points(gv.Points(verts, vdims=['z']))

Now we have the simplices and vertices we can construct the ``TriMesh``. Since the TriMesh is too large to display on its own we apply the ``datashade`` operation, which will rasterize and interpolate the data.

In [ ]:
tiles = gv.WMTS('https://maps.wikimedia.org/osm-intl/{Z}/{X}/{Y}@2x.png')
trimesh = gv.TriMesh((tris, points))
chesapeake_interp = tiles * datashade(trimesh, aggregator=ds.mean('z'))
chesapeake_interp

## Meshes over time

When running environmental simulations such as AdH the mesh is also often accompanied by additional data files representing depths, velocity and error values over time.

We again load a static mesh and project it from its native UTM Zone 11 coordinate system to the Mercator projection.

In [ ]:
fpath = '/Users/philippjfr/Downloads/SanDiego/SanDiego.3dm'
tris, verts = read_3dm_mesh(fpath, skiprows=2)
points = gv.operation.project_points(gv.Points(verts, vdims=['z'], crs=ccrs.UTM(11)))
trimesh = gv.TriMesh((tris, points))
san_diego = tiles * datashade(trimesh, aggregator=ds.mean('z'))
san_diego

Next we load amesh2d .dat file containing Overland Velocity values over the course of a simulation. The ``read_mesh2d`` utility returns a dictionary of dataframes indexed by time.

In [ ]:
fpath = '/Users/philippjfr/Downloads/SanDiego/SanDiego_ovl.dat'
dfs = read_mesh2d(fpath)
dfs[0].head()

To explore this data we will wrap it in a ``DynamicMap`` which returns a TriMesh for each timepoint. First we declare some points containing the positions of the vertices and project those. Now we can make use of the ``add_dimension`` method on those points to add the additional 'Velocity' variable we loaded from the ``.dat`` file. Finally we declare a DynamicMap indexed by time with the keys of the dictionary as values.

In [ ]:
points = gv.operation.project_points(gv.Points((verts.x, verts.y), crs=ccrs.UTM(11)))

def time_mesh(time):
    depth_points = points.add_dimension('Velocity', 0, dfs[time].values[:, 0], vdim=True)
    return gv.TriMesh((tris, depth_points), crs=ccrs.GOOGLE_MERCATOR)

meshes = hv.DynamicMap(time_mesh, kdims='Time').redim.values(Time=sorted(dfs.keys()))

Now we can ``rasterize`` the data by interpolating the 'Velocity' and again plot the data over a map. Since the activated ``holomap='scrubber'`` option above we get a widget to explore the dataset over time.

In [ ]:
%%opts Image [colorbar=True] (cmap=cm_n['rainbow'])
velocity_meshes = rasterize(meshes, aggregator=ds.mean('Velocity'))
tiles * velocity_meshes.redim.range(Velocity=(-0.3, 0.3))